In [1]:
# prerequisite: down data files from kaggle
# https://www.kaggle.com/c/jigsaw-toxic-comment-classification-challenge/data

In [2]:
import torch
import pandas as pd
import numpy as np
from torch.nn import BCEWithLogitsLoss
from transformers import LongformerTokenizerFast, \
LongformerModel, LongformerConfig, Trainer, TrainingArguments, EvalPrediction, AutoTokenizer
from transformers.models.longformer.modeling_longformer import LongformerPreTrainedModel, LongformerClassificationHead
from torch.utils.data import Dataset, DataLoader
import random

/media/hdddisk/anaconda3/envs/nlp/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/media/hdddisk/anaconda3/envs/nlp/lib/python3.11/site-packages/transformers/utils/hub.py:128: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(
2025-04-25 05:07:04.484164: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-25 05:07:04.502428: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the foll

In [11]:
# Train the model
import mlflow
mlflow.set_tracking_uri("https://mlflow.yellowday.day")
mlflow.set_experiment("bert-classify-multi-label-jigsaw-toxic-comment")

2025/04/25 05:12:24 INFO mlflow.tracking.fluent: Experiment with name 'bert-classify-multi-label-jigsaw-toxic-comment' does not exist. Creating a new experiment.


In [4]:
# read the dataframe
insults = pd.read_csv('/media/hdddisk/jigsaw-toxic-comment/train.csv')
insults['labels'] = insults[insults.columns[2:]].values.tolist()
insults = insults[['id','comment_text', 'labels']].reset_index(drop=True)

train_size = 0.001
train_dataset=insults.sample(frac=train_size,random_state=200)
test_dataset=insults.drop(train_dataset.index).sample(frac=train_size,random_state=200).reset_index(drop=True)
train_dataset = train_dataset.reset_index(drop=True)
print(len(train_dataset), len(test_dataset))
train_dataset

160 159


,id,comment_text,labels
0,6725d5a6391e5c77,Goal scored for Portugal \n\nThis could be mil...,"[0, 0, 0, 0, 0, 0]"
1,28ea0d2c61db3137,My mistake someone was vandalizing the page so...,"[0, 0, 0, 0, 0, 0]"
2,4de3d6b966b58ec7,Test card F music \nCeefax music isn't the sam...,"[0, 0, 0, 0, 0, 0]"
3,af602c4c5f1b09bc,""":Meh, I guess I can live with either outcome,...","[0, 0, 0, 0, 0, 0]"
4,9e412a7965873237,"UV is my error, above. I'm told by Kimberly Ja...","[0, 0, 0, 0, 0, 0]"
...,...,...,...
155,98a62f98dacc4553,Please use the articles discussion page.,"[0, 0, 0, 0, 0, 0]"
156,af256c8248ec2d55,I can't make a modification to the infobox of ...,"[0, 0, 0, 0, 0, 0]"
157,edc4512a1fb1bee5,"Dont say that was a bad deletion,this article ...","[0, 0, 0, 0, 0, 0]"
158,02769bc92e9bd063,"""\n\n """"no you're wrong, it's perfect acceptab...","[0, 0, 0, 0, 0, 0]"


In [5]:
# instantiate a Longformer for multilabel classification class
class LongformerForMultiLabelSequenceClassification(LongformerPreTrainedModel):
    """
    We instantiate a class of LongFormer adapted for a multilabel classification task. 
    This instance takes the pooled output of the LongFormer based model and passes it through a classification head. We replace the traditional Cross Entropy loss with a BCE loss that generate probabilities for all the labels that we feed into the model.
    """

    def __init__(self, config):
        super(LongformerForMultiLabelSequenceClassification, self).__init__(config)
        self.num_labels = config.num_labels
        self.longformer = LongformerModel(config)
        self.classifier = LongformerClassificationHead(config)
        self.init_weights()
        
    def forward(self, input_ids=None, attention_mask=None, global_attention_mask=None, 
                token_type_ids=None, position_ids=None, inputs_embeds=None, 
                labels=None):
        
        # create global attention on sequence, and a global attention token on the `s` token
        # the equivalent of the CLS token on BERT models. This is taken care of by HuggingFace
        # on the LongformerForSequenceClassification class
        if global_attention_mask is None:
            global_attention_mask = torch.zeros_like(input_ids)
            global_attention_mask[:, 0] = 1
        
        # pass arguments to longformer model
        outputs = self.longformer(
            input_ids = input_ids,
            attention_mask = attention_mask,
            global_attention_mask = global_attention_mask,
            token_type_ids = token_type_ids,
            position_ids = position_ids
        )
        
        # if specified the model can return a dict where each key corresponds to the output of a
        # LongformerPooler output class. In this case we take the last hidden state of the sequence
        # which will have the shape (batch_size, sequence_length, hidden_size). 
        sequence_output = outputs['last_hidden_state']
        
        # pass the hidden states through the classifier to obtain thee logits
        logits = self.classifier(sequence_output)
        outputs = (logits,) + outputs[2:]
        if labels is not None:
            loss_fct = BCEWithLogitsLoss()
            labels = labels.float()
            loss = loss_fct(logits.view(-1, self.num_labels), 
                            labels.view(-1, self.num_labels))
            outputs = (loss,) + outputs
        
        
        return outputs

In [6]:
# instantiate a class that will handle the data
class Data_Processing(object):
    def __init__(self, tokenizer, id_column, text_column, label_column):
        
        # define the text column from the dataframe
        self.text_column = text_column.tolist()
    
        # define the label column and transform it to list
        self.label_column = label_column
        
        # define the id column and transform it to list
        self.id_column = id_column.tolist()
        
    
# iter method to get each element at the time and tokenize it using bert        
    def __getitem__(self, index):
        comment_text = str(self.text_column[index])
        comment_text = " ".join(comment_text.split())
        # encode the sequence and add padding
        inputs = tokenizer.encode_plus(comment_text,
                                       add_special_tokens = True,
                                       max_length= 3048,
                                       padding = 'max_length',
                                       return_attention_mask = True,
                                       truncation = True,
                                       return_tensors='pt')
        input_ids = inputs['input_ids']
        attention_mask = inputs['attention_mask']
        
        labels_ = torch.tensor(self.label_column[index], dtype=torch.float)
        id_ = self.id_column[index]
        return {'input_ids':input_ids[0], 'attention_mask':attention_mask[0], 
                'labels':labels_, 'id_':id_}
  
    def __len__(self):
        return len(self.text_column) 

batch_size = 2
# create a class to process the training and test data
tokenizer = AutoTokenizer.from_pretrained(
    'allenai/longformer-base-4096', 
    padding = 'max_length',
    truncation=True, 
    max_length = 3048
)
training_data = Data_Processing(
    tokenizer, 
    train_dataset['id'], 
    train_dataset['comment_text'], 
    train_dataset['labels']
)

test_data =  Data_Processing(
    tokenizer, 
    test_dataset['id'], 
    test_dataset['comment_text'], 
    test_dataset['labels']
)

# use the dataloaders class to load the data
dataloaders_dict = {
    'train': DataLoader(training_data, batch_size=batch_size, shuffle=True, num_workers=4),
    'val': DataLoader(test_data, batch_size=batch_size, shuffle=True, num_workers=4)
}

dataset_sizes = {
    'train':len(training_data),
    'val':len(test_data)
}

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [7]:
model = LongformerForMultiLabelSequenceClassification.from_pretrained(
    'allenai/longformer-base-4096',
    gradient_checkpointing=False,
    attention_window = 512,
    num_labels = 6,
    return_dict=True
)

Some weights of LongformerForMultiLabelSequenceClassification were not initialized from the model checkpoint at allenai/longformer-base-4096 and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
    
def multi_label_metrics(predictions, labels):
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(predictions))
    y_pred = np.zeros(probs.shape)
    y_true = labels
    y_pred[np.where(probs >= 0.5)] = 1
    f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average='micro')
    roc_auc = roc_auc_score(y_true, y_pred, average = 'micro')
    accuracy = accuracy_score(y_true, y_pred)
    # define dictionary of metrics to return
    metrics = {
        'f1': f1_micro_average,
        'roc_auc': roc_auc,
        'accuracy': accuracy
    }
    return metrics

# Use the aux EvalPrediction class to obtain prediction labels
def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, 
            tuple) else p.predictions
    result = multi_label_metrics(
        predictions=preds, 
        labels=p.label_ids)
    return result

# define the training arguments
training_args = TrainingArguments(
    output_dir = '/media/hdddisk/bert-classify-multi-label/results',
    num_train_epochs=2,                # Number of training epochs
    per_device_train_batch_size=1,     # Batch size for training
    per_device_eval_batch_size=1,      # Batch size for evaluation
    eval_strategy="steps",             # Evaluate every epoch = "epoch"
    eval_steps=20,                     # Number of steps between evaluations
    save_strategy="steps",             # Save model every epoch
    save_steps=20,
    load_best_model_at_end=True,
    warmup_steps=100,                   # Number of warmup steps for learning rate scheduler
    learning_rate = 2e-5,
    weight_decay=0.01,
    logging_steps = 8,
    logging_dir='/media/hdddisk/bert-classify-multi-label/logs',
    run_name = 'longformer_multilabel_paper_trainer_3048_2e5',
    report_to="mlflow",                # Enable logging to MLflow
)
# instantiate the trainer class and check for available devices
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=training_data,
    eval_dataset=test_data,
    compute_metrics = compute_metrics,
    #data_collator = Data_Processing(),

)
device = 'cuda' if torch.cuda.is_available() else 'cpu'

trainer.train()

Exception ignored in: <function MLflowCallback.__del__ at 0x7005231e8540>
Traceback (most recent call last):
  File "/media/hdddisk/anaconda3/envs/nlp/lib/python3.11/site-packages/transformers/integrations/integration_utils.py", line 1347, in __del__
    self._ml_flow.end_run()
  File "/media/hdddisk/anaconda3/envs/nlp/lib/python3.11/site-packages/mlflow/tracking/fluent.py", line 519, in end_run
    MlflowClient().set_terminated(last_active_run_id, status)
  File "/media/hdddisk/anaconda3/envs/nlp/lib/python3.11/site-packages/mlflow/tracking/client.py", line 2943, in set_terminated
    self._tracking_client.set_terminated(run_id, status, end_time)
  File "/media/hdddisk/anaconda3/envs/nlp/lib/python3.11/site-packages/mlflow/tracking/_tracking_service/client.py", line 945, in set_terminated
    self._log_url(run_id)
  File "/media/hdddisk/anaconda3/envs/nlp/lib/python3.11/site-packages/mlflow/tracking/_tracking_service/client.py", line 919, in _log_url
    run_info = self.store.get_run(

Step,Training Loss,Validation Loss,F1,Roc Auc,Accuracy
20,0.642200,0.654840,0.121739,0.728695,0.018868
40,0.572400,0.550752,0.067797,0.533441,0.056604
60,0.325700,0.223330,0.000000,0.500000,0.905660
80,0.119500,0.149744,0.000000,0.500000,0.905660
100,0.061200,0.129102,0.000000,0.500000,0.905660
120,0.030000,0.123449,0.000000,0.500000,0.905660
140,0.394300,0.123220,0.000000,0.500000,0.905660
160,0.024500,0.120075,0.000000,0.500000,0.905660
180,0.018300,0.126855,0.000000,0.500000,0.905660
200,0.016300,0.127503,0.000000,0.500000,0.905660


2025/04/25 05:28:02 INFO mlflow.system_metrics.system_metrics_monitor: Stopping system metrics monitoring...
2025/04/25 05:28:02 INFO mlflow.system_metrics.system_metrics_monitor: Successfully terminated system metrics monitoring!


🏃 View run longformer_multilabel_paper_trainer_3048_2e5 at: https://mlflow.yellowday.day/#/experiments/5/runs/b6f3090579684e008474b86cf4ae70b7
🧪 View experiment at: https://mlflow.yellowday.day/#/experiments/5


TrainOutput(global_step=320, training_loss=0.20483553875237703, metrics={'train_runtime': 848.8233, 'train_samples_per_second': 0.377, 'train_steps_per_second': 0.377, 'total_flos': 629125400217600.0, 'train_loss': 0.20483553875237703, 'epoch': 2.0})